<a href="https://colab.research.google.com/github/yoerisamwel/code_notes/blob/main/Olympic_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/2/athlete_events.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("athlete_events.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()

+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
| ID|                Name|Sex|Age|Height|Weight|          Team|NOC|      Games|Year|Season|       City|               Sport|               Event|Medal|
+---+--------------------+---+---+------+------+--------------+---+-----------+----+------+-----------+--------------------+--------------------+-----+
|  1|           A Dijiang|  M| 24|   180|    80|         China|CHN|1992 Summer|1992|Summer|  Barcelona|          Basketball|Basketball Men's ...|   NA|
|  2|            A Lamusi|  M| 23|   170|    60|         China|CHN|2012 Summer|2012|Summer|     London|                Judo|Judo Men's Extra-...|   NA|
|  3| Gunnar Nielsen Aaby|  M| 24|    NA|    NA|       Denmark|DEN|1920 Summer|1920|Summer|  Antwerpen|            Football|Football Men's Fo...|   NA|
|  4|Edgar Lindenau Aabye|  M| 34|    NA|    NA|Denmark/Sweden|DEN|1900 Summer|1900|Summ

In [5]:
#Create a temporary view.
df.createOrReplaceTempView('games')

In [8]:
# Get the numover of distinct sports played in 2016
spark.sql("SELECT COUNT(*) FROM games").show()

+--------+
|count(1)|
+--------+
|  271116|
+--------+



In [9]:
# Get the number of distinct sports played in 2012
spark.sql("SELECT COUNT(DISTINCT Sport) FROM games").show()

+---------------------+
|count(DISTINCT Sport)|
+---------------------+
|                   66|
+---------------------+



In [13]:
# Write a subquery to get the distinct sports played in 2016 but not in 2012.
spark.sql("""
SELECT DISTINCT Sport
FROM games
WHERE Year = 2016
AND Sport NOT IN (SELECT DISTINCT Sport FROM games WHERE Year = 2012)
""").show()

+---------+
|min(Year)|
+---------+
|     1896|
+---------+



Question 2: Which sports have been played since the first summer Olympics?

In [ ]:
# Get the year of the earliest Olympics in our data
spark.sql("""
SELECT MIN(Year) FROM games
""").show()

+---------+
|min(year)|
+---------+
|     1896|
+---------+



In [14]:
# Get the sports that were played in the earliest Olympic games in our data.
spark.sql("""
SELECT DISTINCT Sport
FROM games
WHERE Year = (SELECT MIN(Year) FROM games)
""").show()

+-------------+
|        Sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



In [17]:
# Determine which sports were in both the 1896 Olympics and the 2016 Olympics.
spark.sql("""
SELECT DISTINCT Sport
FROM games
WHERE Year = 1896
AND Sport IN (SELECT DISTINCT Sport FROM games WHERE Year = 2016)
""").show()

+-------------+
|        Sport|
+-------------+
|   Gymnastics|
|       Tennis|
|     Swimming|
|    Athletics|
|     Shooting|
|    Wrestling|
|Weightlifting|
|      Cycling|
|      Fencing|
+-------------+



Question 3: Who are the top 5 athletes that won most gold, silver and bronze medals?

In [25]:
# Who are the top 5 Gold medal winners in the Olympics
spark.sql("""
SELECT Name, COUNT(Medal) as TotalGold
FROM games
WHERE Medal = 'Gold'
GROUP BY Name
ORDER BY TotalGold DESC
LIMIT 5
""").show()

+--------------------+---------+
|                Name|TotalGold|
+--------------------+---------+
|Michael Fred Phel...|       23|
|Raymond Clarence ...|       10|
|Frederick Carlton...|        9|
|Larysa Semenivna ...|        9|
|   Mark Andrew Spitz|        9|
+--------------------+---------+



In [26]:
# Who are the top 5 Silver medal winners
spark.sql("""
SELECT Name, COUNT(Medal) as TotalGold
FROM games
WHERE Medal = 'Silver'
GROUP BY Name
ORDER BY TotalGold DESC
LIMIT 5
""").show()


+--------------------+---------+
|                Name|TotalGold|
+--------------------+---------+
|Shirley Frances B...|        6|
|Aleksandr Nikolay...|        6|
|           Yang Yang|        6|
|Aleksandr Vladimi...|        6|
|Mikhail Yakovlevi...|        6|
+--------------------+---------+



In [27]:
# Who are the top 5 Bronze medal winners
spark.sql("""
SELECT Name, COUNT(Medal) as TotalGold
FROM games
WHERE Medal = 'Bronze'
GROUP BY Name
ORDER BY TotalGold DESC
LIMIT 5
""").show()

+--------------------+---------+
|                Name|TotalGold|
+--------------------+---------+
|Harri Tapani Kirv...|        6|
|Franziska van Alm...|        6|
|Merlene Joyce Ott...|        6|
|Heikki Ilmari Sav...|        6|
|Aleksey Yuryevich...|        6|
+--------------------+---------+



Output the gold medal winners to csv

Hint: since we have aggregated our data it is safe to go back to pandas

In [30]:
import pandas as pd



df.toPandas().to_csv('top5Gold.csv')